In [ ]:
# import the libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.svm import SVC
from sklearn.metrics import plot_confusion_matrix
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score

In [ ]:
# read the csv file
data = pd.read_csv()

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
# handle missing values

In [ ]:
# check for outliers using box plot

In [ ]:
# convert the categorical values into numerical values

In [ ]:
# standardise the numerical values

In [ ]:
# split the data in training and testing dataset

# Model training

In [ ]:
# use svm to train the model on training dataset

In [ ]:
# evaluate the SVM model

In [ ]:
# use logistic regression

In [ ]:
# evaluate this model

In [ ]:
# use random forest

In [ ]:
# evaluate this model

In [ ]:
# plot confusion matrix for all 3 models

 Give conclusion, which model is best fit for the data